# Battery RUL Estimation using AI Methods
This notebook performs data preprocessing, EDA, feature engineering, model development, and evaluation.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import tensorflow as tf

# Load dataset
df = pd.read_csv('/mnt/data/Battery_RUL.csv')
df.head()


In [ ]:

# Apply filtering based on constraints
df = df[(df['F1'] > 500) & (df['F1'] < 2500)]
df = df[(df['F5'] > 3.7) & (df['F5'] < 4.1)]
df = df[df['F6'] < 4.05]
df = df[df['F3'] < 7000]

# Fill missing values via linear interpolation
df.interpolate(method='linear', inplace=True)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)


In [ ]:

def exponential_moving_average(data, alpha=0.5):
    result = [data[0]]
    for val in data[1:]:
        result.append(alpha * val + (1 - alpha) * result[-1])
    return result

for col in ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'Total Time(s)']:
    df[col] = exponential_moving_average(df[col].values)


In [ ]:

features = df.columns.drop(['Cycle Index', 'RUL'])
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[features] = scaler.fit_transform(df_scaled[features])


In [ ]:

# Correlation matrix
plt.figure(figsize=(10, 8))
sns.heatmap(df_scaled.corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()


In [ ]:

pca = PCA(n_components=5)
X_pca = pca.fit_transform(df_scaled[features])
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of Components")
plt.ylabel("Explained Variance")
plt.title("PCA - Explained Variance")
plt.grid(True)
plt.show()


In [ ]:

X = df_scaled[features].values.reshape(-1, len(features), 1)
y = df_scaled['RUL'].values

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [ ]:

input_layer = tf.keras.layers.Input(shape=X_train.shape[1:])
x = tf.keras.layers.MultiHeadAttention(num_heads=2, key_dim=32)(input_layer, input_layer)
x = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
output = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=input_layer, outputs=output)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='mse', metrics=['mae'])

history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                    epochs=30, batch_size=32, callbacks=[
                        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
                    ])


In [ ]:

y_pred = model.predict(X_test).flatten()

plt.figure()
plt.plot(y_pred, label='Predicted RUL')
plt.plot(y_test, label='Actual RUL')
plt.legend()
plt.title("Predicted vs Actual RUL")
plt.grid(True)
plt.show()

from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
